In [ ]:
!pip3 install altair vega_datasets --user

# Example Here traffic

To load traffic data, execute
```
bash$ docker-compose -f ./docker/docker-compose-dev.yml exec --user $(id -u) tdmqc /bin/bash -c 'cd ${TDMQ_DIST}/examples/here && fake_user.sh bash ingestor_driver.sh'
```


In [1]:
import altair as alt
import folium
import json
import pandas
import requests
from tdmq.client import Client

from datetime import datetime, timedelta

INFO:tdmq.client.client:Logging is active.


In [2]:
def traffic_chart(s, field, after=None, before=None):
    ts = s.timeseries(after=after, before=before)
    df = pandas.DataFrame({"datetime": ts.time, field: ts.series[field]})
    chart = alt.Chart(df).mark_line().encode(
        x="datetime",
        y=alt.Y(field, 
                scale=alt.Scale(zero=False),
                axis=alt.Axis(title='km/h')))
    chart.properties(width=180, height=180)
    return chart

In [3]:
c = Client()

In [4]:
center = (9.14, 39.25)
radius = 1000
swp_center = (center[1], center[0])

In [5]:
m = folium.Map(location=swp_center, zoom_start=15)

In [6]:
folium.Circle(swp_center, radius=radius).add_to(m)

In [7]:
m

In [8]:
srcs = c.find_sources({'entity_type': 'TrafficObserver', 'roi': f'circle({center}, {radius})'})

In [9]:
len(srcs)

21

In [10]:
for s in srcs:
    chart = traffic_chart(s, 'SU')
    gj = folium.GeoJson(s.default_footprint)
    gj.add_child(folium.Popup().add_child(folium.VegaLite(chart)))
    gj.add_to(m)

Click on one of the roads in blue to get the timeseries of the traffic speed.

In [12]:
m